In [6]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# Load libraries

import pandas
import csv
from numpy import vectorize
from pandas import DataFrame
from pandas.io.parsers import TextFileReader
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors.kde import KernelDensity
import random
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from scipy.sparse import dia_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
import pandas as pd

# read labeld data for user i

testUser = 8
f = open('../Detecting-Cyber-Attacks-Kaggle-Callenge/data/sample_submission.csv', 'r')
reader = csv.reader(f)
mylist = list(reader)
f.close()
res = []
# read labeld data

dataset = pandas.read_csv('../Detecting-Cyber-Attacks-Kaggle-Callenge/data/partial_labels.csv')

# for all

(x, segments) = dataset.axes
array = dataset.values

z = np.reshape(array[testUser, 1:], -1)
z = z.astype('int')
Y_train = z[0:50]
Y_test = z
line = 1
    
for testUser in range(10, 40):
    masquerder = []
    # read instructions

    fileName = array[testUser, 0]
    instructions = pandas.read_fwf('../Detecting-Cyber-Attacks-Kaggle-Callenge/data/FraudedRawData/' + fileName, header=None)
    arr = np.reshape(instructions.values, (150, 100))
    user_test = np.array(arr[0:, :])
    user_train = arr[:50, :]
    hack = []

    for i in range(10):
        labeld_data = np.reshape(array[i, 1:], -1)
        labeld_data = labeld_data.astype('int')
        fileName = array[i, 0]
        instructions = pandas.read_fwf('../Detecting-Cyber-Attacks-Kaggle-Callenge/data/FraudedRawData/' + fileName,
                                       header=None)
        arr = np.reshape(instructions.values, (150, 100))
        for j in range(150):
            if labeld_data[j] == 1:
                hack.append(arr[j, :])

    user_train = np.array(user_train)
    hack = np.array(hack)
    all = np.concatenate((user_train, hack))

    trainCorpus = []
    corpus = []
    test_corpus = []

    for i in range(len(user_test)):
        s = ' '.join(user_test[i, :])
        test_corpus.append(s)

    for i in range(len(all)):
        s = ' '.join(all[i, :])
        if i < 50:
            trainCorpus.append(s)
            corpus.append(s)
        else:
            corpus.append(s)

    vectorizer = CountVectorizer(ngram_range=(2, 2), min_df=0.5)
    vectorizer.fit_transform(trainCorpus)
    train_all = vectorizer.transform(corpus)
    x_test = vectorizer.transform(test_corpus)

    ##################################################################

    wordsCount = {}
    uniqe = set()
    unique_feature_train = np.zeros(150)
    unique_feature_test = np.zeros(150)
    for i in range(len(all)):
        for j in range(100):
            if i < 50:
                uniqe.add(all[i][j])
                unique_feature_train[i] = random.randint(0, 7)
            else:
                if all[i][j] not in uniqe:
                    unique_feature_train[i] = random.randint(12, 20)

    for i in range(len(user_test)):
        if i < 50:
            unique_feature_test[i] = random.randint(0, 10)
        for j in range(100):
            if user_test[i][j] not in uniqe:
                unique_feature_test[i] = unique_feature_test[i] + 1

    ##################################################################

    train_all = np.array(train_all.toarray())
    x_test = np.array(x_test.toarray())
    classes = np.concatenate((np.zeros(50), np.ones(100)))

    train_all = np.c_[train_all, unique_feature_train]
    x_test = np.c_[x_test, unique_feature_test]

    for i in range(len(train_all)):
        print (train_all[i, :])

    clf = SVC(gamma='auto', kernel='linear')
    clf.fit(train_all, classes)

    predicts = clf.predict(x_test)
			

[ 2.  2.  0.  1.  2.  5.  6.  2.  3. 26.  1.  2.  2.  2.  7.]
[5. 5. 3. 1. 5. 1. 2. 6. 3. 1. 2. 0. 1. 4. 3.]
[ 1.  1.  0.  1.  1.  3.  5.  1.  1. 11.  0.  2.  3.  1.  3.]
[1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 5.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
[3. 3. 5. 0. 2. 0. 0. 3. 8. 0. 1. 5. 0. 2. 3.]
[ 3.  3.  4.  1.  3.  2.  5.  3. 13.  1.  0.  9.  5.  4.  3.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 8. 1. 5. 0. 0. 2.]
[ 1.  1.  0.  1.  1.  1.  2.  1. 18. 21.  2.  5.  1.  1.  1.]
[3. 3. 3. 1. 3. 4. 4. 3. 3. 0. 2. 0. 2. 3. 3.]
[4. 4. 5. 0. 4. 0. 0. 4. 6. 0. 1. 9. 0. 4. 5.]
[ 3.  3.  1.  2.  3.  5.  6.  3.  1. 19.  4.  4.  3.  3.  6.]
[4. 4. 2. 1. 4. 4. 4. 5. 2. 0. 0. 4. 2. 4. 2.]
[ 0.  0.  1.  0.  0.  0.  2.  0.  2. 12.  0.  2.  1.  0.  4.]
[1. 1. 0. 1. 1. 1. 2. 1. 8. 7. 0. 4. 1. 1. 0.]
[4. 4. 4. 1. 4. 4. 4. 4. 3. 0. 1. 2. 1. 4. 1.]
[ 2.  2.  0.  2. 

[2. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 2.
 0. 0. 2. 0. 0. 2. 3. 5.]
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 3. 4. 0. 0. 0. 0. 0. 0. 0. 3.
 0. 1. 3. 0. 0. 2. 1. 1.]
[2. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.
 0. 0. 3. 0. 0. 2. 3. 4.]
[2. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 2.
 0. 1. 2. 0. 0. 2. 3. 2.]
[2. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 4.
 0. 0. 3. 0. 0. 2. 2. 6.]
[3. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 7.
 0. 0. 3. 0. 0. 4. 5. 6.]
[ 1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  7. 13.  0.  0.
  0.  0.  0.  0.  0.  2.  0.  9.  1.  0.  0.  1.  1.  3.]
[4. 0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 4.
 0. 1. 4. 0. 0. 4. 5. 2.]
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 5. 8. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 2. 1. 0. 0. 1. 1. 7.]
[3. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 3. 3. 0. 0. 0. 0. 0. 0. 0. 5.
 0. 

[0. 0. 0. 0. 0. 3. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 1. 0.
 1. 1. 0. 0. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 5. 0. 1.
 0. 5. 2. 1. 1. 6.]
[1. 2. 1. 1. 1. 2. 3. 1. 0. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 0. 0. 1.
 2. 0. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 0.
 2. 2. 0. 0. 0. 3.]
[2. 4. 2. 2. 2. 0. 3. 2. 0. 2. 2. 2. 2. 2. 2. 1. 2. 2. 1. 1. 2. 0. 0. 1.
 0. 0. 1. 2. 1. 0. 0. 2. 2. 2. 2. 1. 3. 2. 1. 2. 2. 2. 2. 3. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 4.]
[1. 2. 1. 1. 1. 0. 5. 1. 0. 1. 1. 1. 1. 1. 4. 2. 1. 1. 2. 1. 1. 2. 1. 1.
 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 2. 3. 1. 1. 1. 1. 1. 1. 3. 0. 0. 2. 0.
 1. 0. 0. 0. 0. 3.]
[1. 2. 0. 1. 1. 3. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1.
 2. 0. 1. 1. 1. 0. 2. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 4. 0. 1.
 1. 4. 2. 1. 1. 4.]
[2. 4. 2. 2. 2. 0. 5. 2. 0. 2. 1. 2. 1. 2. 3. 2. 1. 1. 1. 2. 1. 0. 0. 2.
 0. 0. 2. 2. 2. 0. 0. 1. 1. 2. 1. 1. 2. 1. 2. 2. 2. 2. 2. 2. 0. 0. 1. 0.
 2. 0. 0. 0. 0. 1.]
[1. 

[2. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 1. 3. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 2. 0. 4. 0. 0. 0. 4.]
[2. 0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 4. 0. 3. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 1. 2. 4. 0. 0. 0. 7.]
[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 3. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 2. 1. 2. 0. 1. 2. 4.]
[0. 1. 1. 3. 0. 0. 0. 2. 1. 1. 1. 1. 0. 1. 0. 0. 0. 2. 0. 4. 1. 1. 1. 2.
 0. 0. 1. 2. 0. 0. 1. 0. 1. 1. 0. 0. 0.]
[0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3. 0. 1. 3. 0. 0. 0.
 3. 0. 0. 0. 3. 3. 2. 3. 4. 0. 0. 0. 4.]
[1. 1. 4. 2. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 4. 1. 0. 2. 1. 1. 1.
 2. 1. 1. 1. 3. 2. 1. 2. 3. 0. 1. 2. 7.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 5. 1. 0. 0. 1.
 4. 0. 0. 1. 6. 4. 1. 5. 1. 1. 0. 0. 4.]
[4. 1. 3. 2. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 4. 1. 1. 3. 1. 1. 1.
 0. 1. 1. 1. 0. 0. 2. 0. 4. 0. 1. 1. 6.]
[0. 1. 0. 4. 1. 1. 1. 3. 1. 1. 1. 2. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 3.
 0. 1. 1. 3. 0.

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 2. 0. 0. 0. 0. 4.]
[0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 3.]
[1. 0. 1. 3. 0. 0. 1. 1. 0. 2. 1. 0. 1. 0. 0. 0. 0. 0. 1. 3. 2. 0. 1. 1.
 2. 5. 1. 3. 1. 0. 3.]
[1. 1. 1. 3. 1. 1. 1. 0. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 1. 0. 0.
 2. 1. 1. 3. 0. 1. 7.]
[2. 0. 3. 6. 0. 0. 1. 1. 0. 4. 2. 0. 2. 0. 0. 0. 0. 0. 2. 0. 2. 0. 2. 1.
 2. 2. 2. 6. 1. 0. 5.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 7.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. 13.  0.  0.  0.  0.  0. 13.  0.  0.  0.  0.  1.]
[ 1.  1.  1.  3.  1.  1.  1.  0.  1.  2.  1.  1.  1.  1.  1.  1.  1.  1.
  1. 14.  1.  1.  0.  0.  1.  0.  1.  3.  0.  1.  2.]
[1. 1. 1. 3. 1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 8. 3. 1. 1. 2.
 3. 1. 1. 3. 2. 1. 1.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 

...

[14.  3.]
[0. 5.]
[12.  6.]
[27.  0.]
[59.  0.]
[99.  2.]
[51.  2.]
[1. 6.]
[53.  4.]
[99.  3.]
[99.  0.]
[15.  5.]
[6. 5.]
[0. 7.]
[11.  7.]
[77.  1.]
[81.  6.]
[87.  6.]
[99.  5.]
[99.  6.]
[99.  0.]
[99.  2.]
[99.  2.]
[99.  5.]
[99.  7.]
[99.  1.]
[82.  5.]
[99.  3.]
[78.  6.]
[99.  6.]
[95.  6.]
[99.  4.]
[99.  6.]
[99.  6.]
[99.  3.]
[99.  5.]
[99.  5.]
[99.  7.]
[86.  3.]
[82.  7.]
[13.  2.]
[9. 3.]
[0. 5.]
[8. 7.]
[0. 4.]
[0. 7.]
[0. 4.]
[0. 7.]
[0. 3.]
[88.  7.]
[ 0. 17.]
[0. 0.]
[ 0. 14.]
[0. 0.]
[ 0. 16.]
[ 0. 15.]
[ 0. 17.]
[ 0. 12.]
[ 0. 19.]
[ 0. 15.]
[ 0. 16.]
[ 0. 19.]
[0. 0.]
[ 0. 17.]
[ 0. 17.]
[ 0. 17.]
[ 0. 20.]
[ 0. 15.]
[ 0. 16.]
[ 0. 16.]
[ 0. 20.]
[ 0. 16.]
[ 0. 15.]
[ 0. 18.]
[ 0. 13.]
[ 0. 19.]
[ 0. 15.]
[ 0. 19.]
[ 0. 18.]
[ 0. 18.]
[ 0. 13.]
[ 0. 14.]
[ 0. 12.]
[ 0. 15.]
[ 0. 16.]
[ 0. 18.]
[ 0. 17.]
[0. 0.]
[ 0. 15.]
[ 0. 19.]
[ 0. 17.]
[ 0. 15.]
[ 0. 14.]
[ 0. 17.]
[ 0. 19.]
[ 0. 12.]
[ 0. 12.]
[ 0. 18.]
[ 0. 18.]
[ 0. 17.]
[ 0. 16.]
[ 0. 20.]
[ 0. 18.]
[ 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 6.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 2.]
[8. 1. 8. 9. 0. 7. 6. 1. 6. 5. 2. 3.]
[11.  7.  6.  2.  5.  5. 10.  5.  4.  5.  2.  5.]
[14.  6.  7.  7.  6. 14.  6.  6.  6.  6.  7.  0.]
[13.  6.  6.  7.  7. 13.  7.  7.  7.  7.  7.  7.]
[10.  4.  9.  3.  0.  2. 10.  0.  8. 10.  2.  7.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 4.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 6.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
[3. 1. 3. 5. 1. 2. 2. 1. 3. 3. 1. 0.]
[9. 4. 7. 4. 2. 4. 9. 2. 6. 5. 1. 6.]
[14.  6.  7.  5.  6. 11.  7.  6.  5.  6.  5.  0.]
[14.  6.  7.  7.  6. 14.  6.  6.  7.  6.  7.  1.]
[10.  7.  9.  4.  4.  7.  9.  4.  8.  9.  4.  4.]
[7. 

[2. 2. 0. 2. 0. 0. 0. 2. 3. 1. 1. 0. 0. 1. 0. 7.]
[0. 1. 0. 3. 0. 1. 2. 1. 4. 0. 1. 1. 0. 2. 0. 3.]
[3. 5. 0. 4. 1. 3. 2. 3. 5. 4. 2. 2. 0. 1. 2. 0.]
[ 0.  0.  0.  3.  1. 10.  3.  0.  0.  0.  0.  1.  2.  2.  2.  3.]
[3. 2. 0. 4. 0. 4. 6. 0. 0. 0. 0. 4. 3. 2. 1. 7.]
[ 0.  1.  2.  6.  5.  8. 12.  0.  4.  0.  0.  8.  3.  4.  3.  0.]
[0. 0. 0. 2. 0. 3. 5. 1. 1. 1. 1. 3. 2. 1. 2. 1.]
[0. 0. 0. 2. 0. 1. 2. 1. 2. 0. 0. 1. 1. 1. 1. 3.]
[ 4.  5.  3.  5.  3.  1. 10.  2. 10.  1.  0.  7.  3.  3.  1.  2.]
[ 3.  2.  2.  4.  0.  2.  3.  6. 28.  1.  1.  3.  0.  2.  1.  3.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
[1. 2. 0. 4. 1. 2. 3. 2. 2. 4. 3. 2. 1. 0. 2. 3.]
[ 0.  0.  1.  4.  5.  7. 20.  1.  1.  1.  1. 12.  7.  2.  8.  6.]
[ 5.  1.  1.  4.  2. 13. 12.  2.  2.  2.  2. 13.  0.  2.  7.  4.]
[3. 3. 1. 4. 1. 5. 7. 2. 2. 3. 3. 4. 3. 3. 2. 5.]
[0. 5. 3. 1. 0. 3. 9. 4. 7. 6. 4. 5. 4. 2. 1. 2.]
[ 0.  0.  2.  1.  2.  0. 14.  1. 11.  2.  1.  7.  6.  1.  0.  0.]
[ 0.  1.  2.  0.  1.  0.  5.  4. 19.  

[2. 2.]
[3. 1.]
[0. 4.]
[0. 1.]
[0. 7.]
[0. 4.]
[0. 2.]
[1. 2.]
[8. 4.]
[7. 7.]
[2. 1.]
[2. 5.]
[1. 6.]
[5. 3.]
[1. 2.]
[1. 2.]
[1. 5.]
[6. 6.]
[2. 4.]
[1. 4.]
[3. 5.]
[0. 7.]
[0. 4.]
[3. 2.]
[1. 2.]
[0. 5.]
[2. 0.]
[2. 5.]
[0. 3.]
[0. 4.]
[2. 7.]
[0. 2.]
[0. 7.]
[0. 5.]
[0. 3.]
[11.  0.]
[8. 6.]
[2. 7.]
[0. 0.]
[1. 4.]
[2. 5.]
[0. 1.]
[0. 6.]
[0. 1.]
[5. 4.]
[2. 5.]
[0. 3.]
[0. 4.]
[0. 3.]
[4. 4.]
[ 0. 18.]
[0. 0.]
[0. 0.]
[0. 0.]
[ 0. 12.]
[ 0. 19.]
[ 0. 16.]
[ 0. 15.]
[ 0. 13.]
[ 0. 12.]
[ 0. 12.]
[ 0. 16.]
[0. 0.]
[ 0. 16.]
[ 0. 16.]
[ 0. 15.]
[ 0. 20.]
[ 0. 14.]
[ 0. 18.]
[ 0. 16.]
[ 0. 19.]
[ 0. 17.]
[ 0. 13.]
[ 0. 18.]
[ 0. 18.]
[ 0. 12.]
[0. 0.]
[ 0. 17.]
[ 0. 17.]
[ 0. 20.]
[ 0. 17.]
[ 0. 12.]
[ 0. 14.]
[ 0. 14.]
[ 0. 16.]
[ 0. 17.]
[ 0. 15.]
[0. 0.]
[ 0. 14.]
[ 0. 17.]
[ 0. 16.]
[ 0. 14.]
[ 0. 13.]
[ 0. 19.]
[ 0. 19.]
[ 0. 13.]
[ 0. 16.]
[ 0. 16.]
[ 0. 15.]
[ 0. 18.]
[ 0. 14.]
[ 0. 18.]
[ 0. 15.]
[ 0. 20.]
[ 0. 20.]
[ 1. 12.]
[ 0. 14.]
[ 0. 14.]
[ 0. 13.]
[ 0. 12.]
[ 0. 15.]


ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [7]:
from datetime import date
import os
results_df = pd.DataFrame(columns=["id","label"])
#curr_user_results = predicts[50:150].astype(int)
curr_user_results_df = pd.DataFrame(columns=["id","label"])

start_rows = 5000
till_rows = 5100

for testUser in range(10, 40):
    start_rows = 5000
    till_rows = 5100
    for i in range(curr_user_results.size):
        curr_user_results_df.loc[i] = ["User{}_{}-{}".format(testUser,start_rows,till_rows),curr_user_results[i]]
        start_rows = start_rows + 100
        till_rows = till_rows + 100
    results_df = results_df.append(curr_user_results_df)

fname = "submission_results_{}.csv".format(date.today())
results_df.to_csv(fname, index=False)

In [ ]:
os.system('kaggle competitions submit -c detectingcyberattackscallenge -f "{0}" -m "{1}"'.format(fname, "Submitting results to kaggle"))